## 通过Prompt进行文本改写

In [3]:
import os, openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        n=3,
        presence_penalty=0,
        frequency_penalty=2,
    )
    return response

long_text = """
在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。
还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。
"""
short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

version 1: 在现代社会中，我们面对各种挑战和困难，包括外部因素如经济萧条、全球变暖和自然灾害以及内部因素如情感问题、健康问题和自我怀疑。要克服这些困难，我们需要积极应对并具备坚韧不拔的意志、创造性思维以及寻求外部支持的能力。只有这样才能实现潜力并取得成功。
version 2: 现代社会中，我们面临各种挑战和困难，有外部因素如经济、自然灾害等；也有内部因素如情感、健康问题等。要克服这些困难，需要积极应对和寻求支持，并具备坚韧不拔的意志和创造性思维。只有这样才能实现潜力并取得成功。
version 3: 现代生活充满挑战，有外部因素如经济困难和自然灾害，也有内部问题如情感、健康和自我怀疑等。我们需要积极应对这些问题，坚韧不拔并寻求支持来实现成功。


## 通过Logit_bias过滤关键词

In [5]:
import tiktoken
encoding = tiktoken.get_encoding('p50k_base')
token_ids = encoding.encode("灾害")
print(token_ids)

bias_map = {}
for token_id in token_ids:
    bias_map[token_id] = -100
    
# token_ids = encoding.encode("怀疑")
# print(token_ids)
# for token_id in token_ids:
#     bias_map[token_id] = 100


def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, max_tokens=2048,
        n=3, presence_penalty=0, frequency_penalty=2, 
        logit_bias = bias_map,
    )
    return response


short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

[163, 223, 122, 22522, 111]
version 1: 现代社会中，我们面临各种挑战和困难示。有些是外部因素引起的，如经济萧条、自然災室等；还有一些是内部因素引起的，如情感问题、健康问题等。要克服这些挑战和困随需具备坚韧意志、创造性思维及寻求外部支持能力。只有这样才能实现潜力并获得成功。
version 2: 现代社会中，我们遇到各种挑战和困难示例：经济萧条、全球变暖、情感问题等。要克服这些，需要积枪的态度和行动力，并寻求外部支持。只有如此才能实现自己的潜力并获得成功。
version 3: 在现代社会中，我们面临着各种挑战和困难示例：经济萧条、全球变暖、情感问题等。为了克服这些问题，我们需要积枝主动态度和行动，并具备坚韧不拔的意志和创造性思维。同时也要学会寻求外部支持来实现自己的潜力并取得成功。


In [9]:
encoding = tiktoken.get_encoding('gpt2')
token_ids = encoding.encode("灾害")
print(token_ids)


[163, 223, 122, 22522, 111]


In [4]:
def translate(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个翻译，把用户的话翻译成英文"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, 
        max_tokens=2048, top_p=1
    )
    return response["choices"][0]["message"]["content"]

chinese = long_text
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))
print(english)
print(f"chinese: {num_of_tokens_in_chinese} tokens")
print(f"english: {num_of_tokens_in_english} tokens")


In this fast-paced modern society, each of us faces various challenges and difficulties. Some of these challenges and difficulties are caused by external factors, such as economic downturns, global warming, and natural disasters. There are also some that are caused by internal factors, such as emotional issues, health problems, and self-doubt. To overcome these challenges and difficulties, we need to adopt a positive attitude and take action. This means we must possess a resilient will and creative thinking, as well as the ability to seek external support. Only then can we truly realize our potential and achieve success.
chinese: 432 tokens
english: 119 tokens


## 列出所有支持的模型

In [10]:
import pandas as pd
# list all open ai models
engines = openai.Engine.list()
pd = pd.DataFrame(openai.Engine.list()['data'])
display(pd[['id', 'owner']])

,id,owner
0,whisper-1,openai-internal
1,babbage,openai
2,gpt-3.5-turbo,openai
3,davinci,openai
4,text-davinci-edit-001,openai
5,text-davinci-003,openai-internal
6,babbage-code-search-code,openai-dev
7,text-similarity-babbage-001,openai-dev
8,code-davinci-edit-001,openai
9,text-davinci-001,openai


In [11]:
print(pd.head())

   object                     id  ready            owner permissions created
0  engine              whisper-1   True  openai-internal        None    None
1  engine                babbage   True           openai        None    None
2  engine          gpt-3.5-turbo   True           openai        None    None
3  engine                davinci   True           openai        None    None
4  engine  text-davinci-edit-001   True           openai        None    None


## 看看模型维度数量

In [12]:
from openai.embeddings_utils import get_embedding

text = "让我们来算算Embedding"

embedding_ada = get_embedding(text, engine="text-embedding-ada-002")
print("embedding-ada: ", len(embedding_ada))

similarity_ada = get_embedding(text, engine="text-similarity-ada-001")
print("similarity-ada: ", len(similarity_ada))

embedding-ada:  1536
similarity-ada:  1024


In [7]:
babbage_similarity = get_embedding(text, engine="babbage-similarity")
print("babbage-similarity: ", len(babbage_similarity))

babbage_search_query = get_embedding(text, engine="text-search-babbage-query-001")
print("search-babbage-query: ", len(babbage_search_query))

babbage-similarity:  2048
search-babbage-query:  2048


In [8]:
curie = get_embedding(text, engine="curie-similarity")
print("curie-similarity: ", len(curie))

curie-similarity:  4096


In [13]:
davinci = get_embedding(text, engine="text-similarity-davinci-001")
print("davinci-similarity: ", len(davinci))

davinci-similarity:  12288


## Inserting Text接口

In [10]:

prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """\n面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

def insert_text(prefix, suffix):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prefix,
        suffix=suffix,
        max_tokens=1024,
        )
    return response

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

另一些则来自内部，如缺乏资源、失去动力和压力过高的工作等。



In [12]:

prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

然而，大多数的挑战和困难都来自于我们自身内在的心理因素，例如内向的性格、保守的价值观、负面的思维模式和低自信等。
要


## Moderate接口

In [13]:
def chatgpt(text):
    messages = []
    messages.append( {"role": "system", "content": "You are a useful AI assistant"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
    )
    message = response["choices"][0]["message"]["content"]
    return message

threaten = "你不听我的我就拿刀砍死你"
print(chatgpt(threaten))

对不起，我是一个人工智能助手，我不会感觉痛苦或死亡。同时，我是为了帮助您解决问题而设计的，如果您有任何需要帮助的事情，请告诉我，我会尽力帮助您。


In [14]:
threaten = "你不听我的我就拿刀砍死你"

def moderation(text):
    response = openai.Moderation.create(
        input=text
    )
    output = response["results"][0]
    return output
print(moderation(threaten))

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": true,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 0.039922066032886505,
    "hate/threatening": 0.00029030031873844564,
    "self-harm": 0.0026187379844486713,
    "sexual": 3.374317748239264e-05,
    "sexual/minors": 2.1676822115068717e-08,
    "violence": 0.9995468258857727,
    "violence/graphic": 4.8206060455413535e-05
  },
  "flagged": true
}
